This notebook is intended to demonstrate how vessel segmentation methods of ITKTubeTK can be applied to multi-channel MRI (MRA + T1, T2, etc).

In [3]:
import itk
from itk import TubeTK as ttk

import numpy as np

In [4]:
ImageType = itk.Image[itk.F, 3]
im1iso = itk.imread("results/CT-Lungs.mha")
im1MaskedVess = itk.imread("results/CT-Lungs-Masked-VesselEnhanced.mha")

In [5]:
imMath = ttk.ImageMath.New(im1MaskedVess)
imMath.MedianFilter(1)
imMath.Threshold(0.000001, 1, 1, 0)
im1VessMask = imMath.GetOutputShort()

ccSeg = ttk.SegmentConnectedComponents.New(im1VessMask)
ccSeg.SetMinimumVolume(10)
ccSeg.Update()
im1VessMaskCC = ccSeg.GetOutput()

itkImageF4 not loaded from module TubeTK because of exception:
 module 'itk.TubeTKPython' has no attribute 'itkImageF4'
vectoritkImageF4 not loaded from module TubeTK because of exception:
 module 'itk.TubeTKPython' has no attribute 'vectoritkImageF4'


In [8]:
imMathSS = ttk.ImageMath.New(im1VessMaskCC)
imMathSS.Threshold(0,0,1,0)
im1VessMaskInv = imMathSS.GetOutputFloat()

distFilter = itk.DanielssonDistanceMapImageFilter.New(im1VessMaskInv)
distFilter.Update()
dist = distFilter.GetOutput()

imMath.SetInput(dist)
imMath.Blur(0.4)
tmp = imMath.GetOutput()
imMath.ReplaceValuesOutsideMaskRange(tmp, 0.1, 10, 0)
im1SeedRadius = imMath.GetOutput()

itk.imwrite(im1SeedRadius, "results/Lungs-VesselsSeedRadius.mha")

In [10]:
imMath.SetInput(im1iso)
imMath.ReplaceValuesOutsideMaskRange(im1MaskedVess, 0, 1000, 0)
imMath.Blur(0.4)
imMath.IntensityWindow(0.5,1000,0,1000)
im1Input = imMath.GetOutput()

itk.imwrite(im1iso, "results/Lungs-VesselsInput.mha")

<IPython.core.display.Javascript object>

In [ ]:
numSeeds = 40

vSeg = ttk.SegmentTubes.New(Input=im1Input)
#vSeg.SetVerbose(True)
vSeg.SetMinCurvature(0)#.0001)
vSeg.SetMinRoundness(0.02)
vSeg.SetMinRidgeness(0.5)
vSeg.SetMinLevelness(0.0)
vSeg.SetRadiusInObjectSpace( 0.5 )
vSeg.SetBorderInIndexSpace(3)
vSeg.SetSeedMask( im1SeedRadius )
vSeg.SetSeedRadiusMask( im1SeedRadius )
vSeg.SetOptimizeRadius(False)
vSeg.SetUseSeedMaskAsProbabilities(True)
vSeg.SetSeedExtractionMinimumProbability(0.6)
#vSeg.SetSeedMaskMaximumNumberOfPoints( numSeeds )
vSeg.ProcessSeeds()

In [13]:
tubeMaskImage = vSeg.GetTubeMaskImage()

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

In [14]:
SOWriter = itk.SpatialObjectWriter[3].New()
SOWriter.SetInput(vSeg.GetTubeGroup())
SOWriter.SetBinaryPoints(True)
SOWriter.SetFileName( "results/Lungs-Vessels.tre" )
SOWriter.Update()